In [1]:
import sys
sys.path.append('/home/punygod_admin/SoundSense/soundsense')

In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
from torch.autograd import Variable
import cv2
import os
import numpy as np
import yaml
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models.baselines.mulsa.inference import MULSAInference
from models.imi_datasets import ImitationEpisode

/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-26 14:36:10.899968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model_path = "/home/punygod_admin/SoundSense/soundsense/models/baselines/mulsa/lightning_logs/sorting_imi_vg_ag_lstm_seqlen_3_spec04-22-21:39:20/last.ckpt"
config_path = '/home/punygod_admin/SoundSense/soundsense/models/baselines/mulsa/lightning_logs/sorting_imi_vg_ag_lstm_seqlen_3_spec04-22-21:39:20/hparams.yaml'
with open(config_path) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
model = MULSAInference(config_path)

# Load the model from the checkpoint
model.load_state_dict(
        torch.load(
            model_path,
            map_location=torch.device("cpu"),
        )['state_dict']
    )
# model = model.load_from_checkpoint(model_path)

# Then, move the model to CUDA device
model = model.cuda()

/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


Using GPU
Creating LSTM decoder
Total parameters: 29.449992 Million
NUM STACK:  6


In [5]:

np.random.seed(0)
run_ids = os.listdir('/home/punygod_admin/SoundSense/soundsense/data/mulsa/sorting')
np.random.permutation(run_ids)
train_val_split = 0.8
split = int(train_val_split*len(run_ids))
train_episodes = run_ids[:split]
val_episodes = run_ids[split:]

print("Train episodes: ", len(train_episodes))
print("Val episodes: ", len(val_episodes))
val_set = torch.utils.data.ConcatDataset(
    [
        ImitationEpisode(config, run_id, train=False)
        for run_id in val_episodes
    ]
)

val_loader = DataLoader(val_set, num_workers=config["num_workers"], shuffle=False,batch_size=1)

Train episodes:  79
Val episodes:  20
Loading audio for ng/87
Loading audio for ng/94
Loading audio for ng/42
Loading audio for ng/98
Loading audio for ng/86
Loading audio for ng/80
Loading audio for ng/48
Loading audio for ng/16
Loading audio for ing/4
Loading audio for ng/97
Loading audio for ng/71
Loading audio for ng/78
Loading audio for ng/17
Loading audio for ng/83
Loading audio for ing/7
Loading audio for ng/69
Loading audio for ng/19
Loading audio for ng/56
Loading audio for ing/9
Loading audio for ng/88


In [6]:
for i, (img, target) in enumerate(val_loader):
    img[0] = img[0].squeeze(0)
    img[0] = img[0].permute(0, 2, 3, 1)
    print(img[1].squeeze(0).shape)
    inp = {"video": np.array(img[0]), "audio": img[1].squeeze(0).to(device)}  

    # Forward pass through the model
    output = model(inp)
    output = output[0][-1].reshape(1,-1)
    print(output.data.shape)
    _, predicted_class = torch.max(output.data, 1)

    if predicted_class != 0:
        continue

    # Compute the gradients of the predicted class output with respect to the feature maps
    model.zero_grad()
    output[:, predicted_class].backward()


    # Get the gradients from the last convolutional layer
    v_grads, a_grads = model.get_activations_gradient()
    print("a_grads: ", a_grads.shape) # torch.Size([6, 512, 5, 7])

    # Get the feature maps from the last convolutional layer
    v_maps, a_maps = model.get_activations()
    print("a_maps: ", a_maps.shape) # torch.Size([6, 512, 5, 7])

    # Perform global average pooling on the gradients
    a_pooled_grads = torch.mean(a_grads, dim=(2, 3))
    print("a_pooled_grads: ", a_pooled_grads.shape) # torch.Size([6, 512])

    # Multiply each feature map by its corresponding gradient value
    for i in range(a_maps.shape[1]):
        for j in range(a_maps.shape[0]):
            a_maps[j, i, :, :] *= a_pooled_grads[j, i]
    print("v_maps: ", a_maps.shape) # torch.Size([6, 512, 5, 7])
    
    # Obtain the heatmap by averaging the weighted feature maps
    a_heatmap = torch.mean(a_maps, dim=1).squeeze().cpu()
    print("v_heatmap: ", a_heatmap.shape) # torch.Size([6, 5, 7])

    # Normalize the heatmap
    a_heatmap = torch.maximum(a_heatmap, torch.zeros_like(a_heatmap))
    print("a_heatmap: ", a_heatmap.shape) # torch.Size([6, 5, 7])
    # print(type(v_heatmap)) # <class 'torch.Tensor'>
    # print(torch.max(torch.max(v_heatmap, dim=1)[0], dim=1)[0]) 

    a_heatmap_max = torch.max(torch.max(a_heatmap, dim=1)[0], dim=1)[0] # torch.Size([6])
    for i in range(a_heatmap.shape[0]):
        a_heatmap[i] /= a_heatmap_max[i]

    print("v_heatmap: ", a_heatmap.shape) # torch.Size([6, 5, 7])
    
    # Resize the heatmap to match the input image size
    img[0] = img[0].permute(0, 3, 1, 2)
    print("img[0].shape: ", img[0].shape) # torch.Size([6, 3, 75, 100])


    a_heatmap = a_heatmap.numpy()
    a_heatmap_resized = np.zeros((a_heatmap.shape[0], img[0].shape[2], img[0].shape[3])) # (6, 75, 100)

    for i in range(a_heatmap.shape[0]):
        a_heatmap_resized[i] = cv2.resize(a_heatmap[i], (img[0].shape[3], img[0].shape[2])) # (75, 100)

    a_heatmap_resized = (255 * a_heatmap_resized).astype(np.uint8) 
    a_heatmap_resized = np.minimum(a_heatmap_resized, 255)
    print("a_heatmap_resized: ", a_heatmap_resized.shape) # (6, 75, 100)   

    # Apply the heatmap to the input image
    superimposed_img_a = np.zeros((a_heatmap.shape[0], img[0].shape[2], img[0].shape[3], 3))
    for i in range(a_heatmap.shape[0]):
        temp_heatmap = cv2.applyColorMap(a_heatmap_resized[i], cv2.COLORMAP_JET) / 255
        # print(min(temp_heatmap.flatten()), max(temp_heatmap.flatten())) # 0 1
        temp_img = img[0][i].cpu().numpy().transpose(1, 2, 0)*0.5 + 0.5
        # print(min(temp_img.flatten()), max(temp_img.flatten())) # 0 1
        superimposed_img_a[i] = temp_heatmap * 0.3 + temp_img * 0.7

    # Display the input image and the GradCAM heatmap with colorbar
    fig, axs = plt.subplots(6, 2, figsize=(10, 30))
    for i in range(6):
        axs[i, 0].imshow(img[0][i].cpu().numpy().transpose(1, 2, 0)*0.5 + 0.5)
        fig.colorbar(axs[i, 1].imshow(superimposed_img_a[i]), ax=axs[i, 1], cmap='jet')

/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/punygod_admin/miniconda3/envs/muls/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([1, 64, 501])


NameError: name 'device' is not defined